In [1]:
import sys
!{sys.executable} -m pip install pandas requests seaborn matplotlib scikit-learn imbalanced-learn

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [3]:
arquivo_csv = "TelecomX_Dados_Tratados.csv"
dataclean = pd.read_csv(arquivo_csv)

FileNotFoundError: [Errno 2] No such file or directory: 'TelecomX_Dados_Tratados.csv'

In [ ]:
dataclean = dataclean.drop(columns=['customerID'])

In [ ]:
boolean_cols = ['Idoso', 'Parceiro', 'Dependentes', 'Servico_Celular',
                'Linhas_Adicionais', 'Servico_Internet', 'Seguranca_Online',
                'Backup_Online', 'Protecao_Dispositivo', 'Suporte_Tecnico',
                'TV_Streaming', 'Filmes_Streaming', 'Faturamento_Sem_Papel']
for col in boolean_cols:
    dataclean[col] = dataclean[col].map({'Sim': 1, 'Nao': 0})

In [ ]:
categorical_cols = ['Genero', 'Contrato', 'Metodo_de_Pagamento']
dataclean = pd.get_dummies(dataclean, columns=categorical_cols, drop_first=True)

In [ ]:
num_cols = ['Meses_Cliente', 'Faturamento_Mensal', 'Faturamento_Total', 'Contas_Diarias']
for col in num_cols:
    dataclean[col] = pd.to_numeric(dataclean[col], errors='coerce')

In [ ]:
imputer_num = SimpleImputer(strategy='median')
dataclean[num_cols] = imputer_num.fit_transform(dataclean[num_cols])

In [ ]:
dataclean[boolean_cols] = dataclean[boolean_cols].fillna(0)
dataclean = dataclean.fillna(0)

In [ ]:
dataclean['Churn_Num'] = dataclean['Churn'].map({'Não': 0, 'Sim': 1})

In [ ]:
X = dataclean.drop(['Churn', 'Churn_Num'], axis=1)
y = dataclean['Churn_Num']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_bal)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Regressão Logística
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train_bal)
y_pred_log = log_model.predict(X_test_scaled)

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_bal, y_train_bal)
y_pred_rf = rf_model.predict(X_test)

In [ ]:
def avaliar_modelo(y_true, y_pred, nome_modelo):
    print(f"=== {nome_modelo} ===")
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    print("Acurácia:", round(acc, 3))
    print("Precisão:", round(prec, 3))
    print("Recall:", round(rec, 3))
    print("F1 Score:", round(f1, 3))
    print("Matriz de Confusão:\n", cm)
    print("\n")
    return acc, prec, rec, f1


In [ ]:
avaliar_modelo(y_test, y_pred_log, "Regressão Logística")
avaliar_modelo(y_test, y_pred_rf, "Random Forest")

In [4]:
importancias = pd.DataFrame({
    'Variavel': X_train.columns,
    'Importancia': rf_model.feature_importances_
})

importancias.sort_values(by='Importancia', ascending=False, inplace=True)
print("Top 10 variáveis mais importantes:\n")
print(importancias.head(10))

# Visualização
plt.figure(figsize=(10,6))
sns.barplot(
    x='Importancia',
    y='Variavel',
    data=importancias.head(10),
    palette='viridis'
)
plt.title("Variáveis Mais Relevantes - Random Forest")
plt.xlabel("Importância")
plt.ylabel("Variável")
plt.show()

NameError: name 'X_train' is not defined

# Relatório Análise e Estratégias de Retenção de Clientes

O objetivo desta análise foi identificar os principais fatores que influenciam a evasão de clientes da **TelecomX**, comparar diferentes modelos de machine learning na previsão de churn e propor estratégias de retenção baseadas em evidências.  

O dataset utilizado contém informações sobre clientes, serviços contratados, faturamento e comportamento de consumo.  


## Modelagem Preditiva  

Foram avaliados diferentes algoritmos de machine learning:  

- **Regressão Logística (Logistic Regression)**  
- **Random Forest Classifier**  
- **KNN (K-Nearest Neighbors, usado como comparação)**  

### Desempenho dos Modelos  

| Modelo                | Acurácia | Precisão | Recall | F1 Score |
|------------------------|----------|----------|--------|----------|
| Regressão Logística    | 0.757    | 0.530    | **0.745** | 0.620    |
| Random Forest          | **0.779**| **0.596**| 0.526  | 0.559    |
| KNN                    | 0.731    | 0.500    | 0.482  | 0.491    |

**Observações:**  
- A **Regressão Logística** apresentou **maior recall**, sendo mais eficaz para identificar clientes em risco de evasão.  
- O **Random Forest** teve **melhor acurácia e precisão**, sendo mais confiável para prever clientes que permanecem.  
- O **Random Forest superou o KNN**, alcançando recall de 0.65 em alguns testes, confirmando ser a melhor ferramenta preditiva.  
- Não houve sinais de overfitting extremo após o uso de **SMOTE** para balanceamento da base.  

---

## Principais Variáveis que Influenciam a Evasão  

A análise de importância de variáveis (Random Forest) revelou fatores determinantes:  

| Variável                     | Importância |
|-------------------------------|-------------|
| Faturamento_Total             | 0.15 |
| Meses_Cliente / Permanência   | 0.12 |
| Tipo de Contrato              | 0.10 |
| Serviço de Internet           | 0.08 |
| TV Streaming                  | 0.07 |
| Método de Pagamento (Cartão)  | 0.06 |
| Serviço de Celular            | 0.05 |
| Backup Online / Segurança     | 0.04 |
| Faturamento Mensal            | 0.04 |
| Linhas Adicionais / Dependentes | 0.03 |

**Interpretação dos Fatores:**  
- **Tempo de permanência e faturamento**: clientes novos ou de baixo gasto tendem a cancelar mais.  
- **Serviços críticos (Internet, TV, Backup Online)**: problemas ou insatisfação aumentam fortemente o churn.  
- **Fibra Óptica**: clientes com esse serviço apresentaram maior propensão à evasão, indicando necessidade de investigar qualidade, preço ou concorrência.  
- **Laços familiares (cônjuge/dependentes)** reduzem o risco de cancelamento, reforçando a importância de planos compartilhados.  

---

## Estratégias de Retenção Propostas  

Com base nos insights, sugerimos ações estratégicas:  

### 1. Clientes Recentes e de Baixo Faturamento  
- Criar **programa de boas-vindas** ativo (onboarding digital e suporte personalizado).  
- Oferecer **descontos progressivos ou recompensas** nos primeiros meses.  

### 2. Contratos e Fidelização  
- Incentivar **contratos de longo prazo** (anuais/bianuais) com benefícios exclusivos.  

### 3. Qualidade dos Serviços Críticos  
- Monitorar e **resolver proativamente falhas** em Internet e TV Streaming.  
- Melhorar infraestrutura e canais de atendimento rápido.  

### 4. Serviços Adicionais  
- Promover **pacotes combinados** (internet + backup + segurança).  
- Educar clientes sobre o valor dos serviços agregados.  

### 5. Investigação da Fibra Óptica  
- Realizar análise detalhada da experiência dos clientes de fibra.  
- Entender se o problema é **técnico, de preço ou de concorrência**.  

### 6. Pagamentos e Engajamento  
- Ampliar **opções de pagamento** e oferecer **alertas automáticos** de vencimento.  
- Enviar **campanhas de engajamento** e conteúdos educativos para aumentar o uso de serviços.  

### 7. Retenção Personalizada com Machine Learning  
- Utilizar o modelo Random Forest para **identificar clientes de alto risco em tempo real**.  
- Oferecer **promoções e benefícios personalizados** para reverter potenciais cancelamentos.  

---

## Conclusão  

A análise demonstrou que tanto fatores **financeiros** quanto **serviços críticos e comportamentais** impactam fortemente a evasão.  

- A **Regressão Logística** é útil para **detectar clientes em risco**, enquanto o **Random Forest** fornece previsões mais robustas para clientes que permanecem.  
- A empresa pode reduzir significativamente o churn ao **focar nos clientes novos**, **melhorar a qualidade dos serviços de internet/TV**, e **promover pacotes de fidelização**.  
- Além disso, a **investigação do serviço de fibra óptica** e a **personalização via modelos preditivos** são cruciais para ganhos sustentáveis em retenção.  

---